In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from shapely.geometry.polygon import Polygon
import json
import numpy as np
import re
import itertools
from scipy.special import expit

# Wirkt sich die Geografie beim Autokauf aus?

Beim PKW-Kauf haben Kund:innen eine breite Auswahl, was die Motorisierung angeht. Natürlich gibt es unzählige Faktoren, die den Kauf beeinflussen, doch wir schauen uns einmal spezifisch an, ob die Tatsache, dass man in einer flachen oder bergigen Gegend lebt, sich auf die Wahl des PKW auswirkt. Zum Vergleich ziehen wir die Wirtschaftskraft heran, gemessen am verfügbaren Haushaltseinkommen.

In [ ]:
df = pd.read_csv("data/krs_pkw_rug.csv",
                 dtype = {"krs_code": object},
                 index_col = "krs_code"
                )

geo = gpd.read_file("download/Kreisgrenzen/vg2500_12-31.utm32s.shape/vg2500/VG2500_KRS.shp")
geo = geo[["AGS", "geometry"]]
geo = geo.rename({"AGS": "krs_code"}, axis = 1)
geo = geo.to_crs(epsg = "4236")

gdf = pd.merge(df, geo, on = "krs_code")
gdf = gpd.GeoDataFrame(gdf)
gdf = gdf.set_index("krs_code")

## Motorisierung

Besteht ein Zusammenhang zwischen dem Gelände und der Motorisierung, gemessen am Hubraum des Motors?

Wir schauen zunächst auf die Variablen Haushaltseinkommen und Geländerauheit und ihren Zusammenhang mit der Frage, wie viele der Autos in einer Gegend stärker motorisiert sind. Die Vorbereitung dieser Daten findet in einem anderen Dokument statt `<link>`, weil sie selber einige Zeilen in Anspruch nimmt und von eher technischer Natur ist, während wir hier lieber Zusammenhänge in grafischen Darstellungen und statistischen Modellen suchen. Hier also die Verteilung dieser drei Variablen auf der Deutschlandkarte:

In [ ]:
# Wir wiederholen den Plot später, also automatisieren:
def plot_gdf_choro(zvars, coltitles, hovertexts):
    ncol = len(zvars)
        
    fig = make_subplots(rows = 1, cols = ncol,
                        specs = [ list( itertools.repeat({"type": "choropleth"}, ncol) ) ],
                        column_titles = coltitles,
                        horizontal_spacing = 0.0025 )

    fig.update_layout(margin = dict(t=25, r=0, b=0, l=0),
                      coloraxis_showscale = False,
                      autosize = True,
                      height = 400, width = 400*ncol,
                      paper_bgcolor = "rgba(0,0,0,0)",
                      plot_bgcolor = "rgba(0,0,0,0)"
                     )

    for i in range(ncol):
        zvar, hovertext = (zvars[i], hovertexts[i])
        fig.add_trace(
            go.Choropleth(locations = gdf.index,
                          z = zvar.astype(float), # Data to be color-coded
                          geojson = json.loads(gdf[["geometry"]].to_json()),
                          showscale = False,
                          customdata = gdf.name,
                          hovertemplate = hovertext,
                          marker_line = {"width": .5,
                                         "color": "black"}
                          ),
            col = i+1, row = 1)

    fig.update_geos(fitbounds="locations",
                    visible = False,
                    projection_type = "mercator")
    return(fig)

plot_gdf_choro(zvars = [gdf.m_rugged, gdf.cm3_over_1999_pct, gdf.income],
               coltitles = ["Geländerauheit", "Anteil antriebsstarker PKW", "Pro-Kopf-Einkommen"],
               hovertexts = ["<b>%{customdata}</b><br>Rauhheit: %{z:.1f}<extra></extra>",
                             "<b>%{customdata}</b><br>PKW ab 2.000cm³: %{z:.2f} %<extra></extra>",
                             "<b>%{customdata}</b><br>Einkommen: %{z:.0f} €<extra></extra>"])

Ob zwischen unseren beiden Prädiktoren und der Motorisierung eine Korrelation besteht, lässt sich auch über einen einfachen Scatterplot zeigen. In den beiden Tafeln steigen Rauhheit und Einkommen jeweils von links nach rechts an. Ein Zusammenhang mit der Motorisierung (y-Achse) zeigt sich in einer mehr oder weniger starken Tendenz der Punkte, zu einer Linie zu konvergieren. Spätestens hier deutet sich ein Unterschied zwischen Rauhheit und Einkommen an. Und auch hier gibt die Berührung mit der Maus genaue Zahlen preis.

In [ ]:
def plot_gdf_scatter(xvars, yvar, coltitles, hovertexts):

    fig = make_subplots(rows = 1, cols = 2,
                        column_titles = coltitles,
                        horizontal_spacing = 0.0075,
                        shared_yaxes = True )

    figA = go.Scatter(x = xvars[0],
                      y = yvar,
                      mode = "markers",
                      showlegend = False,
                      marker_opacity = .5)

    figB = go.Scatter(x = xvars[1],
                      y = yvar,
                      mode = "markers",
                      showlegend = False,
                      marker_opacity = .5)

    fig.add_trace(figA, row=1, col=1)
    fig.add_trace(figB, row=1, col=2)

    fig.update_traces(customdata = gdf.name)

    fig.data[0].hovertemplate = hovertexts[0]
    fig.data[1].hovertemplate = hovertexts[1]
    
    fig.update_layout(title = {"text": ""},
                      height = 400, width = 800,
                      margin = dict(t=25, r=5, b=5, l=5),
                      yaxis_ticks = None,
                      plot_bgcolor = "rgba(200,200,200,.3)",
                      paper_bgcolor = "rgba(0,0,0,0)",
                      xaxis_gridcolor = "rgba(0,0,0,0)",
                      xaxis2_gridcolor = "rgba(0,0,0,0)",
                      yaxis_gridcolor = "rgba(0,0,0,0)",
                      yaxis2_gridcolor = "rgba(0,0,0,0)",
                     )
    
    return(fig)

fig = plot_gdf_scatter(yvar = gdf.cm3_over_1999_pct / 100,
                       xvars = [gdf.m_rugged, gdf.income],
                       coltitles = ["Geländerauheit", "Haushaltseinkommen pro Kopf"],
                       hovertexts = ["<b>%{customdata}</b><br>Rauhheit: %{x:.1f}<br>PKW ab 2.000cm³: %{y:.2%}<extra></extra>",
                                     "<b>%{customdata}</b><br>Einkommen: %{x:.1f}<br>PKW ab 2.000cm³: %{y:.2%}<extra></extra>"]
                      )
fig.update_layout(yaxis_title_text = "PKW ab 2.000 cm³ Hubraum",
                  yaxis_tickformat = ".0%")

fig.show()

### Statistisches Modell

Ein lineares Modell kann Aufschluss über die Rolle der beiden Prädiktoren bei der Wahl der Motorisierung geben. Wir benutzen ein Lineares Modell (genauer: ein Bayes'sches Generalized Linear Model), um den Zusammenhang der Variablen Rauhheit und Einkommen mit der Motorisierung zu quantifizieren. Anders als bei der obigen grafischen Darstellung ist es hier einfacher, die Daten aus allen drei Kategorien der Motorisierung zu nutzen. In einem Plot sähe das zu unübersichtlich aus, daher dort nur die Darstellung der höchsten Kategorie.

In [ ]:
%load_ext rpy2.ipython
import rpy2.robjects as robjects

In [ ]:
%%R -i df
library(brms)
library(tidyverse)
library(tidybayes)

# Datensatz vorbereiten:
df <- tibble(df) |>
    transmute(
        id = 1:nrow(df),
        name,
        max_cm3_1399,
        max_cm3_1999,
        cm3_over_1999,
        income,
        income_scaled = scale(income),
        m_rugged,
        m_rugged_scaled = scale(m_rugged),
        size = max_cm3_1399 + max_cm3_1999 + cm3_over_1999
    )

# Priors setzen, Modell formulieren und fitten:
cbind2 <- cbind
prior <- prior(normal(0, 10), "b" ) +
         prior(cauchy(0, 1), "Intercept")

model_ir <- brm(bf( cbind2(max_cm3_1399, max_cm3_1999, cm3_over_1999) | trials(size) ~ income_scaled * m_rugged_scaled + (1 | id) ),
                data = df,
                family = multinomial(),
                prior = prior,
                iter = 5500, warmup = 500, chains = 6, cores = 6,
                file = "data/model_ir.rds")

Wir haben zwei Prädiktoren (Geländerauhheit und Einkommen), die sich auf das Ergebnis auswirken. Ein dreidimensionaler Plot könnte hier theoretisch passen. Doch auch wenn diese Art der Darstellung beliebt ist und Informationsdichte suggeriert, ist sie eher ungeeignet für genaue Schlüsse aus den Daten oder bei nur subtilen Phänomenen. Informationsdichte ist nicht unser Ziel, sondern Verständnis. Wir helfen uns stattdessen mit zwei Perspektiven auf den gleichen Sachverhalt: Zuerst schauen wir uns an, ob die Geländerauheit (x-Achse) zusammenhängt damit, welche Anteile der zugelassenen PKW (y-Achse) in die drei Kategorien für den Hubraum fallen. Und dafür halten wir einmal das Einkommen niedrig und einmal hoch. Ändert sich viel bei der Zusammensetzung der deutschen Wagenflotte von links nach rechts, dann steht die Landschaft tatsächlich im Zusammenhang mit der Motorisierung. Ändert sich eher etwas von der linken zur rechten Tafel, dann spricht das für einen Zusammenhang mit dem Einkommen.

In [ ]:
%%R

# der Wertebereich der beiden Prädiktoren, über den wir Vorhersagen treffen wollen:
simdat <- expand_grid(
    income_scaled = seq(-2, 2, length.out = 10),
    m_rugged_scaled = seq(-2, 2, length.out = 10),
    size = 1e5,
    id = "doesn't matter"
)

post_ir <- simdat |>
    # Modellvorhersagen finden in Anzahlen von Autos statt (wie im Datensatz):
    add_predicted_draws(model_ir,
                        allow_new_levels = TRUE) |>
    mean_qi(.width = c(.95, .50)) |>
    mutate(.category = as.character(.category),
           .row = NULL,
           ) |>
    # in Form bringen:
    pivot_wider(names_from = .category,
                values_from = .prediction:.upper) |>
    transmute(
        income_scaled,
        m_rugged_scaled,
        width = .width,
        low_ratio = .prediction_max_cm3_1399  / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        low_diff_lower = (.prediction_max_cm3_1399 - .lower_max_cm3_1399) / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        low_diff_upper = (.upper_max_cm3_1399 - .prediction_max_cm3_1399) / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        mid_ratio = .prediction_max_cm3_1999  / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        mid_diff_lower = (.prediction_max_cm3_1999 - .lower_max_cm3_1999) / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        mid_diff_upper = (.upper_max_cm3_1999 - .prediction_max_cm3_1999) / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        hi_ratio  = .prediction_cm3_over_1999 / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        hi_diff_lower = (.prediction_cm3_over_1999 - .lower_cm3_over_1999) / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999),
        hi_diff_upper = (.upper_cm3_over_1999 - .prediction_cm3_over_1999) / (.prediction_max_cm3_1399 + .prediction_max_cm3_1999 + .prediction_cm3_over_1999)
    ) |>
    transmute(
        # Spalten mit Zwischenwerten entfernen,
        # Anteile aufeinander stapeln für die Darstellung:
        income_scaled,
        m_rugged_scaled,
        width,
        low_ratio = low_ratio,
        low_lower = low_ratio - low_diff_lower,
        low_upper = low_ratio + low_diff_upper,
        mid_ratio = mid_ratio + low_ratio,
        mid_lower = mid_ratio - mid_diff_lower,
        mid_upper = mid_ratio + mid_diff_upper,
        hi_ratio  = hi_ratio  + mid_ratio,
        hi_lower  = hi_ratio  - hi_diff_lower,
        hi_upper  = hi_ratio  + hi_diff_upper,
    ) |>
    # in Form bringen:
    pivot_longer(cols = low_ratio:hi_upper,
                 names_sep = "_",
                 names_to = c("motor", "stat")) |>
    # shape to [income x rugged x width x motor, stat]:
    pivot_wider(names_from = stat,
                values_from = value)

colnames_post <- colnames(post_ir)

In [ ]:
post_ir = pd.DataFrame( robjects.globalenv["post_ir"] ).transpose()
post_ir.columns = robjects.globalenv["colnames_post"]

In [ ]:
colors = ["rgba(255,237,160,1)", "rgba(254,178,76,1)", "rgba(240,59,32,1)"]
motors = ["low", "mid", "hi"]
annotations = ["kleine Motoren", "mittlere Motoren", "große Motoren"]
incomes = [-2.0, 2.0]
widths = [.95, .5]

fig = make_subplots(rows = 1, cols = 2,
                    horizontal_spacing = .02,
                    column_titles = ["Geringes Einkommen", "Hohes Einkommen"], )

for plotcolumn, income_scaled in zip([1,2], incomes):
    # filter to df for each panel:
    post_ir_thispanel = post_ir.loc[
        (post_ir.income_scaled == income_scaled)
    ]

    for color, motor, annotation in zip(colors, motors, annotations):
        # filter to df for each trace:
        post_ir_thistrace = post_ir_thispanel.loc[
            (post_ir.motor == motor) &
            (post_ir.width == .95) # arbitrary; need any 1 width to draw mean
        ]
        this_annotation_y = np.mean(post_ir_thistrace.ratio) - .05

        # the area trace:
        fig.add_trace(
            go.Scatter(
                x = post_ir_thistrace.m_rugged_scaled,
                y = post_ir_thistrace.ratio,
                mode = "lines",
                fill = "tonexty",
                line = dict(width = 1.5, color = "white"),
                fillcolor = color,
                showlegend = False,
            ), col = plotcolumn, row = 1 )
        
        # motorization label:
        fig.add_annotation(x = .0,
                           y = this_annotation_y,
                           text = annotation,
                           xref = "x",
                           xanchor = "center",
                           showarrow = False,
                           font = dict(color = "rgba(0,0,0,1)",
                                       size = 14),
                           row = 1, col = plotcolumn
                          )

    # depict uncertainty:
    uncertainty_hovertexts = [
        "95%-Unsicherheitsintervall: Die Kategoriengrenze verläuft mit 95%iger Wahrscheinlichkeit in diesem Intervall.",
        "50%-Unsicherheitsintervall: Die Kategoriengrenze verläuft mit 50%iger Wahrscheinlichkeit in diesem Intervall."
    ]
    for width, uncertainty_hovertext in zip(widths, uncertainty_hovertexts):
        for motor in ["low", "mid"]:
            post_ir_uncertainty = post_ir_thispanel.loc[(post_ir_thispanel.width == width) &
                                                        (post_ir_thispanel.motor == motor)]
            fig.add_trace(
                go.Scatter(
                    x = pd.concat([post_ir_uncertainty.m_rugged_scaled,
                                   post_ir_uncertainty.m_rugged_scaled[::-1]]),
                    y = pd.concat([post_ir_uncertainty.upper,
                                   post_ir_uncertainty.lower[::-1]]),
                    mode = "lines",
                    line = dict(width = 0),
                    fill = "toself",
                    fillcolor = "rgba(255, 255, 255, .3)",
                    showlegend = False,
                ),
                row = 1, col = plotcolumn
            )

    # arrow:
    fig.add_annotation(x = 1.5, xref = "x",
                       y = .1, yref = "y",
                       ax = -1.5, axref = "x"+str(plotcolumn),
                       ay = .1, ayref = "y",
                       text = "", arrowhead = 2, arrowcolor = "rgba(0,0,0,1)", arrowsize = 2,
                       row = 1, col = plotcolumn
                      )
    
    # Bergland & Flachland:
    fig.add_annotation(x = 2, y = .05, showarrow = False,
                       text = "Bergland", xanchor = "right",
                       font = dict(color = "rgba(0,0,0,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )
    fig.add_annotation(x = -2, y = .05, showarrow = False,
                       text = "Flachland", xanchor = "left",
                       font = dict(color = "rgba(0,0,0,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )
                  
    
fig.update_layout(height = 400,
                  width = 800,
                  xaxis = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  xaxis2 = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  yaxis = dict(tickvals = np.arange(0, 1.1, .1),
                               tickformat = ".0%",
                               range = [0,1],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  yaxis2 = dict(tickvals = [],
                               range = [0,1],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  margin = dict(t=50, r=20, b=20, l=20),
                  plot_bgcolor = "rgba(200,200,200,.3)",
                  paper_bgcolor = "rgba(0,0,0,0)",
                  xaxis_gridcolor = "rgba(0,0,0,0)",
                 )

fig.layout.annotations[0].xanchor = "left"
fig.layout.annotations[0].x = 0
fig.layout.annotations[1].xanchor = "left"
fig.layout.annotations[1].x = .51
    
fig.show()

Es sieht so aus, als würde nicht viel passieren zwischen Flach- und Bergland. Tatsächlich scheint bei geringem Einkommen der Anteil hoch motorisierter PKW sogar leicht abzunehmen. Allerdings haben wir auch zwei Unsicherheitsintervalle um jede Kategoriengrenze herum. Das äußere sagt aus, "zu 95% liegt die Kategoriengrenze hier", und das innere drückt eine 50%ige Sicherheit aus. Diese Intervalle relativieren dann doch stark die scheinbar paradoxe Beobachtung von weniger stark motorisierten Autos im Bergland.

Was eher ins Auge fällt, ist ein gewisser Unterschied zwischen den Tafeln -- bei hohem Einkommen scheint es mehr starke PKW zu geben. Das können wir deutlicher zeigen, indem wir die gleichen Modellvorhersagen mit vertauschten Achsen zeigen:

In [ ]:
colors = ["rgba(255,237,160,1)", "rgba(254,178,76,1)", "rgba(240,59,32,1)"]
motors = ["low", "mid", "hi"]
annotations = ["kleine Motoren", "mittlere Motoren", "große Motoren"]
ruggeds = [-2.0, 2.0]
widths = [.95, .5]

fig = make_subplots(rows = 1, cols = 2,
                    horizontal_spacing = .02,
                    column_titles = ["Flachland", "Bergland"], )

for plotcolumn, rugged_scaled in zip([1,2], ruggeds):
    # filter to df for each panel:
    post_ir_thispanel = post_ir.loc[
        (post_ir.m_rugged_scaled == rugged_scaled)
    ]

    for color, motor, annotation in zip(colors, motors, annotations):
        # filter to df for each trace:
        post_ir_thistrace = post_ir_thispanel.loc[
            (post_ir.motor == motor) &
            (post_ir.width == .95) # arbitrary; need any 1 width to draw mean
        ]
        this_annotation_y = np.mean(post_ir_thistrace.ratio) - .05

        # the area trace:
        fig.add_trace(
            go.Scatter(
                x = post_ir_thistrace.income_scaled,
                y = post_ir_thistrace.ratio,
                mode = "lines",
                fill = "tonexty",
                line = dict(width = 1.5, color = "white"),
                fillcolor = color,
                showlegend = False,
            ), col = plotcolumn, row = 1 )
        
        # motorization label:
        fig.add_annotation(x = .0,
                           y = this_annotation_y,
                           text = annotation,
                           xref = "x",
                           xanchor = "center",
                           showarrow = False,
                           font = dict(color = "rgba(0,0,0,1)",
                                       size = 14),
                           row = 1, col = plotcolumn
                          )

    # depict uncertainty:
    uncertainty_hovertexts = [
        "95%-Unsicherheitsintervall: Die Kategoriengrenze verläuft mit 95%iger Wahrscheinlichkeit in diesem Intervall.",
        "50%-Unsicherheitsintervall: Die Kategoriengrenze verläuft mit 50%iger Wahrscheinlichkeit in diesem Intervall."
    ]
    for width, uncertainty_hovertext in zip(widths, uncertainty_hovertexts):
        for motor in ["low", "mid"]:
            post_ir_uncertainty = post_ir_thispanel.loc[(post_ir_thispanel.width == width) &
                                                        (post_ir_thispanel.motor == motor)]
            fig.add_trace(
                go.Scatter(
                    x = pd.concat([post_ir_uncertainty.income_scaled,
                                   post_ir_uncertainty.income_scaled[::-1]]),
                    y = pd.concat([post_ir_uncertainty.upper,
                                   post_ir_uncertainty.lower[::-1]]),
                    mode = "lines",
                    line = dict(width = 0),
                    fill = "toself",
                    fillcolor = "rgba(255, 255, 255, .3)",
                    showlegend = False,
                ),
                row = 1, col = plotcolumn
            )

    # arrow:
    fig.add_annotation(x = 1.5, xref = "x",
                       y = .1, yref = "y",
                       ax = -1.5, axref = "x"+str(plotcolumn),
                       ay = .1, ayref = "y",
                       text = "", arrowhead = 2, arrowcolor = "rgba(0,0,0,1)", arrowsize = 2,
                       row = 1, col = plotcolumn
                      )
    
    # Geringes & Hohes Einkommen:
    fig.add_annotation(x = 2, y = .05, showarrow = False,
                       text = "Hohes Einkommen", xanchor = "right",
                       font = dict(color = "rgba(0,0,0,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )
    fig.add_annotation(x = -2, y = .05, showarrow = False,
                       text = "Geringes Einkommen", xanchor = "left",
                       font = dict(color = "rgba(0,0,0,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )


fig.update_layout(height = 400,
                  width = 800,
                  xaxis = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  xaxis2 = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  yaxis = dict(tickvals = np.arange(0, 1.1, .1),
                               tickformat = ".0%",
                               range = [0,1],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  yaxis2 = dict(tickvals = [],
                               range = [0,1],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  margin = dict(t=50, r=20, b=20, l=20),
                  plot_bgcolor = "rgba(200,200,200,.3)",
                  paper_bgcolor = "rgba(0,0,0,0)",
                  xaxis_gridcolor = "rgba(0,0,0,0)",
                 )

fig.layout.annotations[0].xanchor = "left"
fig.layout.annotations[0].x = 0
fig.layout.annotations[1].xanchor = "left"
fig.layout.annotations[1].x = .51
    
fig.show()

Hier wird dann deutlich: Der Einfluss des Einkommens auf die Motorisierung ist stärker als der der Landschaft.

## Allradantrieb

Vielleicht die naheliegendste Eigenschaft eines PKW, die in hügeligen Gegenden sinnvoll ist, ist der Allradantrieb. Es folgt ein Blick auf den Zusammenhang zwischen diesem Feature und den bekannten Prädiktoren.

In [ ]:
plot_gdf_choro(zvars = [gdf.m_rugged, gdf.allrad_pct, gdf.income],
               coltitles = ["Geländerauheit", "Anteil PKW mit Allradantrieb", "Pro-Kopf-Einkommen"],
               hovertexts = ["<b>%{customdata}</b><br>Rauheit: %{z:.1f}<extra></extra>",
                             "<b>%{customdata}</b><br>PKW mit Allrad: %{z:.2f} %<extra></extra>",
                             "<b>%{customdata}</b><br>Einkommen: %{z:.0f} €<extra></extra>"])

In [ ]:
fig = plot_gdf_scatter(yvar = gdf.allrad_pct / 100,
                       xvars = [gdf.m_rugged, gdf.income],
                       coltitles = ["Geländerauheit", "Haushaltseinkommen pro Kopf"],
                       hovertexts = ["<b>%{customdata}</b><br>Rauhheit: %{x:.1f}<br>PKW ab 2.000cm³: %{y:.2%}<extra></extra>",
                                     "<b>%{customdata}</b><br>Einkommen: %{x:.1f}<br>PKW ab 2.000cm³: %{y:.2%}<extra></extra>"]
                      )
fig.update_layout(yaxis_title_text = "PKW mit Allradantrieb",
                  yaxis_tickformat = ".0%")

fig.show()

### Statistisches Modell

In [ ]:
%%R -i df
library(BH)
options(browser = 'firefox')

# Datensatz vorbereiten:
df <- tibble(df) |>
    transmute(
        id = 1:nrow(df),
        name,
        allrad,
        income,
        income_scaled = scale(income),
        m_rugged,
        m_rugged_scaled = scale(m_rugged),
        pkw_gesamt
    )

# Priors setzen, Modell formulieren und fitten:
prior <- prior(normal(0, 10), "b" ) +
         prior(cauchy(0, 1), "Intercept")

model_ir_allrad <- brm(bf( allrad | trials(pkw_gesamt) ~ income_scaled * m_rugged_scaled + (1 | id) ),
                       data = df,
                       family = binomial(),
                       prior = prior,
                       iter = 5500, warmup = 500, chains = 6, cores = 6,
                       file = "data/model_ir_allrad.rds")

In [ ]:
%%R

# der Wertebereich der beiden Prädiktoren, über den wir Vorhersagen treffen wollen:
simdat <- expand_grid(
    income_scaled = seq(-2, 2, length.out = 10),
    m_rugged_scaled = seq(-2, 2, length.out = 10),
    pkw_gesamt = 1e5,
    id = "doesn't matter"
)

post_ir_allrad <- simdat |>
    add_predicted_draws(model_ir_allrad,
                        allow_new_levels = TRUE) |>
    mean_qi(.width = c(.95, .50)) |>
    transmute(
        income_scaled,
        m_rugged_scaled,
        width = .width,
        ratio = .prediction / pkw_gesamt,
        lower = .lower / pkw_gesamt,
        upper = .upper / pkw_gesamt
    )

colnames_post <- colnames(post_ir_allrad)

In [ ]:
post_ir_allrad = pd.DataFrame( robjects.globalenv["post_ir_allrad"] ).transpose()
post_ir_allrad.columns = robjects.globalenv["colnames_post"]

In [ ]:
incomes = [-2.0, 2.0]
widths = [.95, .5]

fig = make_subplots(rows = 1, cols = 2,
                    horizontal_spacing = .02,
                    column_titles = ["Geringes Einkommen", "Hohes Einkommen"], )

for plotcolumn, income_scaled in zip([1,2], incomes):
    # filter to df for each panel:
    post_ir_allrad_thispanel = post_ir_allrad.loc[
        post_ir_allrad.income_scaled == income_scaled
    ]
    post_ir_allrad_thistrace = post_ir_allrad_thispanel.loc[
        post_ir_allrad_thispanel.width == .95
    ]

    # depict uncertainty first, line on top:
    for width in widths:
        post_ir_allrad_uncertainty = post_ir_allrad_thispanel.loc[(post_ir_allrad_thispanel.width == width)]
        fig.add_trace(
            go.Scatter(
                x = pd.concat([post_ir_allrad_uncertainty.m_rugged_scaled,
                               post_ir_allrad_uncertainty.m_rugged_scaled[::-1]]),
                y = pd.concat([post_ir_allrad_uncertainty.upper,
                               post_ir_allrad_uncertainty.lower[::-1]]),
                mode = "lines",
                line = dict(width = 0),
                fill = "toself",
                fillcolor = "rgba(255,255,255,.2)",
                showlegend = False,
            ),
            row = 1, col = plotcolumn
        )
        
    fig.add_trace(
    go.Scatter(
        x = post_ir_allrad_thistrace.m_rugged_scaled,
        y = post_ir_allrad_thistrace.ratio,
        mode = "lines",
        # fill = "tozeroy",
        line = dict(width = 1.5, color = "rgba(240,59,32,1)"),
        # fillcolor = color,
        showlegend = False,
    ), col = plotcolumn, row = 1 )


    # arrow:
    fig.add_annotation(x = 1.5, xref = "x",
                       y = .03, yref = "y",
                       ax = -1.5, axref = "x"+str(plotcolumn),
                       ay = .03, ayref = "y",
                       text = "", arrowhead = 2, arrowcolor = "rgba(255,255,255,1)", arrowsize = 2,
                       row = 1, col = plotcolumn
                      )
    
    # Bergland & Flachland:
    fig.add_annotation(x = 2, y = .015, showarrow = False,
                       text = "Bergland", xanchor = "right",
                       font = dict(color = "rgba(255,255,255,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )
    fig.add_annotation(x = -2, y = .015, showarrow = False,
                       text = "Flachland", xanchor = "left",
                       font = dict(color = "rgba(255,255,255,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )
                  
    
fig.update_layout(height = 400,
                  width = 800,
                  xaxis = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  xaxis2 = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  yaxis = dict(tickvals = np.arange(0, 1.1, .05),
                               tickformat = ".0%",
                               range = [0,.3],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)",
                               title_text = "Anteil PKW mit Allrad"
                              ),
                  yaxis2 = dict(tickvals = [],
                               range = [0,.3],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  margin = dict(t=50, r=20, b=20, l=20),
                  plot_bgcolor = "rgba(200,200,200,.3)",
                  paper_bgcolor = "rgba(0,0,0,0)",
                  xaxis_gridcolor = "rgba(0,0,0,0)",
                 )

# fig.layout.annotations[0].xanchor = "left"
# fig.layout.annotations[0].x = 0
# fig.layout.annotations[1].xanchor = "left"
# fig.layout.annotations[1].x = .51
    
fig.show()

In [ ]:
rugged_levels = [-2.0, 2.0]
widths = [.95, .5]

fig = make_subplots(rows = 1, cols = 2,
                    horizontal_spacing = .02,
                    column_titles = ["Flachland", "Bergland"], )

for plotcolumn, m_rugged_scaled in zip([1,2], rugged_levels):
    # filter to df for each panel:
    post_ir_allrad_thispanel = post_ir_allrad.loc[
        post_ir_allrad.m_rugged_scaled == m_rugged_scaled
    ]
    post_ir_allrad_thistrace = post_ir_allrad_thispanel.loc[
        post_ir_allrad_thispanel.width == .95
    ]

    # depict uncertainty first, line on top:
    for width in widths:
        post_ir_allrad_uncertainty = post_ir_allrad_thispanel.loc[(post_ir_allrad_thispanel.width == width)]
        fig.add_trace(
            go.Scatter(
                x = pd.concat([post_ir_allrad_uncertainty.income_scaled,
                               post_ir_allrad_uncertainty.income_scaled[::-1]]),
                y = pd.concat([post_ir_allrad_uncertainty.upper,
                               post_ir_allrad_uncertainty.lower[::-1]]),
                mode = "lines",
                line = dict(width = 0),
                fill = "toself",
                fillcolor = "rgba(255,255,255,.2)",
                showlegend = False,
            ),
            row = 1, col = plotcolumn
        )
        
    fig.add_trace(
    go.Scatter(
        x = post_ir_allrad_thistrace.income_scaled,
        y = post_ir_allrad_thistrace.ratio,
        mode = "lines",
        # fill = "tozeroy",
        line = dict(width = 1.5, color = "rgba(240,59,32,1)"),
        # fillcolor = color,
        showlegend = False,
    ), col = plotcolumn, row = 1 )


    # arrow:
    fig.add_annotation(x = 1.5, xref = "x",
                       y = .03, yref = "y",
                       ax = -1.5, axref = "x"+str(plotcolumn),
                       ay = .03, ayref = "y",
                       text = "", arrowhead = 2, arrowcolor = "rgba(255,255,255,1)", arrowsize = 2,
                       row = 1, col = plotcolumn
                      )
    
    # Bergland & Flachland:
    fig.add_annotation(x = 2, y = .015, showarrow = False,
                       text = "Hohes Einkommen", xanchor = "right",
                       font = dict(color = "rgba(255,255,255,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )
    fig.add_annotation(x = -2, y = .015, showarrow = False,
                       text = "Niedriges Einkommen", xanchor = "left",
                       font = dict(color = "rgba(255,255,255,.7)", size = 16),
                       row = 1, col = plotcolumn
                      )
                  
    
fig.update_layout(height = 400,
                  width = 800,
                  xaxis = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  xaxis2 = dict(tickvals = [],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  yaxis = dict(tickvals = np.arange(0, 1.1, .05),
                               tickformat = ".0%",
                               range = [0,.3],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)",
                               title_text = "Anteil PKW mit Allrad"
                              ),
                  yaxis2 = dict(tickvals = [],
                               range = [0,.3],
                               zeroline = False,
                               gridcolor = "rgba(0,0,0,0)"
                              ),
                  margin = dict(t=50, r=20, b=20, l=20),
                  plot_bgcolor = "rgba(200,200,200,.3)",
                  paper_bgcolor = "rgba(0,0,0,0)",
                  xaxis_gridcolor = "rgba(0,0,0,0)",
                 )

# fig.layout.annotations[0].xanchor = "left"
# fig.layout.annotations[0].x = 0
# fig.layout.annotations[1].xanchor = "left"
# fig.layout.annotations[1].x = .51
    
fig.show()

In [ ]:
%%R
scales <- c(
    attr(df$income_scaled, "scaled:center"),
    attr(df$income_scaled, "scaled:scale"),
    attr(df$m_rugged_scaled, "scaled:center"),
    attr(df$m_rugged_scaled, "scaled:scale")
)

estimates <- fixef(model_ir_allrad)[,1]

In [ ]:
i_c, i_s, r_c, r_s = robjects.globalenv["scales"]
fixef_Intercept, fixef_i, fixef_r, fixef_ir = robjects.globalenv["estimates"]

plane = pd.DataFrame(np.array(np.meshgrid( np.linspace(-2,2,11), np.linspace(-1.2,2,11) ) ).reshape(2,121) ).transpose()
plane.columns = ["income_scaled", "m_rugged_scaled"]
plane["income"] = plane.income_scaled*i_s + i_c
plane["m_rugged"] = plane.m_rugged_scaled*r_s + r_c
plane["estimate"] = expit( fixef_Intercept + fixef_i*plane.income_scaled + fixef_r*plane.m_rugged_scaled + fixef_ir*plane.income_scaled*plane.m_rugged_scaled )

In [ ]:
fig = px.scatter_3d(gdf,
                    x = "m_rugged",
                    y = "income",
                    z = "allrad_pct",
                    opacity = .3)
fig.update_traces(marker_size = 2)
fig.update_layout(width = 800,
                  height = 400,
                  paper_bgcolor = "rgba(0,0,0,0)",
                  scene = dict(
                      xaxis = dict(
                          showbackground = False,
                          gridcolor = "rgba(100,100,100,.1)"),
                      yaxis = dict(
                          showbackground = False,
                          gridcolor = "rgba(100,100,100,.1)"),
                      zaxis = dict(
                          showbackground = False,
                          gridcolor = "rgba(100,100,100,.1)")))

fig.show()